# Step 0: We import the required libraries 

In [1113]:
import pandas as pd
import ast 
import numpy as np
import csv

# Step 1: We import the data exported from Doctolib  

In [1114]:
data1 = pd.read_csv('data/extracted_data_final_0_1000.csv')
data2 = pd.read_csv('data/extracted_data_final1000_10000.csv')
data3 = pd.read_csv('data/extracted_data_final10000_20000.csv')
data4 = pd.read_csv('data/extracted_data_final20000_60000.csv')
data5 = pd.read_csv('data/extracted_data_final60000_100000.csv')
data6 = pd.read_csv('data/extracted_data_final100000_140000.csv')

In [1115]:
data=pd.concat([data1,data2,data3,data4,data5,data6],ignore_index=True)

# Step 2: We reorganize the data exported from Doctolib 

In [1116]:
# Function to reorganize the profile cards 
def reorder_cards(profile_cards):
    try:
        cards_dict = ast.literal_eval(profile_cards)
        
        reordered_cards = {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

        def split_content(value):
            value = value.replace('\\n', '\n').replace('\\', '\n')
            return value.split('\n')

        for key, value in cards_dict.items():
            if value.startswith("Carte et informations d'accès"):
                reordered_cards['card_1'] = split_content(value)
            elif value.startswith("Présentation"):
                reordered_cards['card_2'] = split_content(value)
            elif value.startswith("Horaires et coordonnées"):
                reordered_cards['card_3'] = split_content(value)
            elif value.startswith("Tarifs et remboursement") or value.startswith("Lieu"):
                reordered_cards['card_0'] = split_content(value)
            elif value.startswith("Tarifs"):
                reordered_cards['card_4'] = split_content(value)
            elif value.startswith("Informations légales"):
                reordered_cards['card_5'] = split_content(value)
            else :
                if isinstance(value,list) or isinstance(value,str):
                    reordered_cards['card_0'] = split_content(value)
        
        return reordered_cards
    except:
        return {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

reordered_cards = data['Profile Cards'].apply(reorder_cards)

reordered_cards_df = pd.json_normalize(reordered_cards)

data_combined = pd.concat([data, reordered_cards_df], axis=1)

data_combined.drop(columns = 'Profile Cards', inplace=True)


In [1117]:
#To provide a clearer localisation
def extract_lat_lng(map_location):
    try:
        location_dict = ast.literal_eval(map_location)
        return (location_dict['lat'], location_dict['lng'])
    except:
        return (None, None)

data_combined['Latitude'], data_combined['Longitude'] = zip(*data_combined['Map Location'].apply(extract_lat_lng))

data_combined.drop(columns = 'Map Location', inplace=True)


In [1118]:
data_combined['Skills']=data_combined['Skills'].astype(str)

In [1119]:
# Function to treat the column 'Skills'
def process_skills(skills):
    if skills == 'Skills missing':
        return np.nan
    else:
        return skills.split('\n')

data_combined['Skills Processed'] = data_combined['Skills'].apply(process_skills)

data_combined.drop(columns = 'Skills', inplace=True)

In [1120]:
#To check which specialty has more doctors
data_combined['Specialty'].value_counts()

Specialty
Médecin généraliste         18116
Ostéopathe                  13861
Chirurgien-dentiste         13145
Psychologue                  8730
Masseur-kinésithérapeute     6916
                            ...  
Planning familial - IVG         1
Cancérologue médicale           1
Neurophysiologie                1
Mammographie                    1
Médiatrice en santé             1
Name: count, Length: 210, dtype: int64

In [1121]:
data_combined['RPPS'] = data_combined['card_5'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else np.nan)
data_combined.drop(columns = 'card_5', inplace = True)

In [1122]:
data_combined['Nb Skills'] = data_combined['Skills Processed'].apply(lambda x: len(x)-1 if isinstance(x, list) else 0)

In [1123]:
def paiement(card0):
    if isinstance(card0, list):
        if 'Moyens de paiement' in card0:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['moyens_paiement'] = data_combined['card_0'].apply(paiement)


def vitale(card0):
    if isinstance(card0, list):
        if 'Carte Vitale acceptée' in card0:
            return 1
        elif 'Carte Vitale non acceptée' in card0:
            return 0
        else:
            return np.nan
    else:
        return np.nan
data_combined['carte_vitale'] = data_combined['card_0'].apply(paiement)


def secteur(card0):
    if isinstance(card0, list):
        a=0
        for x in card0:
            if 'Conventionné secteur 2' in x or 'Secteur 2' in x:
                a=2
            elif 'Conventionné secteur 1' in x or 'Secteur 1' in x:
                a=1
            elif 'Conventionné secteur 3' in x or 'Secteur 3' in x:
                a=3
            elif 'Conventionné' in x:
                a=4
        return a
    else:
        return 0
data_combined['secteur'] = data_combined['card_0'].apply(secteur)

In [1124]:
pd.set_option('display.max_columns',None)

In [1125]:
data[data['Doctor Name']=='Dr Michel SARFATI']

,Doctor Name,Specialty,Map Location,Skills,Profile Cards,URL
135469,Dr Michel SARFATI,Médecin généraliste,Map location missing,Skills missing,"{'card_0': ""Présentation\nLe médecin généralis...",https://www.doctolib.fr/medecin-generaliste/vi...


In [1126]:
cards_dict = ast.literal_eval(data['Profile Cards'].iloc[137696])

In [1127]:
data_combined['Présentation']=''
data_combined['Langues parlées']=''
data_combined['Diplômes nationaux et universitaires']=''
data_combined['Autres formations']=''
data_combined['Expériences']=''
data_combined['Travaux et publications']=''
data_combined['Prix et distinctions']=''
data_combined['site']=0

liste_mots_clefs = ['Présentation','Langues parlées','Diplômes nationaux et universitaires','Formations','Autres formations','Expériences', 'Site web', 'Travaux et publications', 'Prix et distinctions']

for i in range(data_combined.shape[0]):
    if isinstance(data_combined['card_2'].iloc[i],list) : 
        liste_card2 = data_combined['card_2'].iloc[i][1:]
        sous_liste = ''
        label = liste_mots_clefs[0]
        for x in liste_card2 :
            if x in liste_mots_clefs :
                data_combined.loc[i, label] = sous_liste
                label = x
                sous_liste = ''
            elif x=='Voir le site':
                data_combined.loc[i, 'site'] = 1
            elif x=='▾ Voir plus':
                None
            else :
                sous_liste = sous_liste+(x)

In [1128]:
data_combined['Diplômes nationaux et universitaires b']=data_combined['Diplômes nationaux et universitaires'].apply(lambda x: 1 if len(x)>0 else 0)

In [1129]:
data_combined['Nb caractères présentation'] = data_combined['Présentation'].apply(lambda x: len(x))

In [1130]:
data_combined['Autres formations b']=data_combined['Autres formations'].apply(lambda x: 1 if len(x)>0 else 0)

In [1131]:
data_combined['Travaux et publications b']=data_combined['Travaux et publications'].apply(lambda x: 1 if len(x)>0 else 0)

In [1132]:
data_combined['Nb langues']=0

for i in range(data_combined.shape[0]):
    if isinstance(data_combined['Langues parlées'].iloc[i],str) : 
        liste_langue = (data_combined['Langues parlées'].iloc[i]).split()
        count = 0
        for x in liste_langue :
            
            if  x != 'et'  :
                count +=1
        data_combined.loc[i,'Nb langues']=count

In [1133]:
data_combined['Expériences b']=data_combined['Expériences'].apply(lambda x: 1 if len(x)>0 else 0)

In [1134]:
data_combined['Formations b']=data_combined['Formations'].apply(lambda x: 1 if isinstance(x,str) else 0)

In [1135]:
liste1 = ["Carte et informations d'accès",'Moyens de transport','Parking public','Informations pratiques',"J'autorise le traitement d'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus", 'sur la collecte et le traitement des données par Google','AFFICHER LA CARTE']
def carac1(card1):
    nb=0
    for phrase in card1:
        if  phrase not in liste1:
            nb += len(phrase)
    return nb
data_combined['Carac_card_1'] = data_combined['card_1'].apply(lambda x: carac1(x)if isinstance(x, list) else 0)

In [1136]:
def transport(card1):
    if isinstance(card1, list):
        if 'Moyens de transport' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Transport'] = data_combined['card_1'].apply(transport)

In [1137]:
def parking(card1):
    if isinstance(card1, list):
        if 'Parking public' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Parking'] = data_combined['card_1'].apply(parking)

In [1138]:
def infos(card1):
    if isinstance(card1, list):
        if 'Informations pratiques' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Infos'] = data_combined['card_1'].apply(infos)

In [1139]:
def coordonnées(card3):
    if isinstance(card3, list):
        if 'Coordonnées' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Coordonnées'] = data_combined['card_3'].apply(coordonnées) 

In [1140]:
def sansRDV(card3):
    if isinstance(card3, list):
        if 'Consultations sans rendez-vous' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Sans RDV'] = data_combined['card_3'].apply(sansRDV)

In [1141]:
nbs=['0','1','2','3','4','5','6','7','8','9','.']
def traitement_tarif(x):
    p1='0'
    p2='0'
    i=0
    while i<len(x) and x[i] not in nbs: #we look for the beginning of the first price
        i+=1
    while i<len(x) and x[i] in nbs: #we isolate the first price
        p1=p1+x[i]
        i+=1
    while i<len(x) and x[i] not in nbs: #we look for the beginning of the second price
        i+=1
    while i<len(x) and x[i] in nbs: #we isolate the second price
        p2=p2+x[i]
        i+=1
    p1=float(p1)
    p2=float(p2)
    if p1==0 and p2==0:
        return []
    elif p1==0:
        return [p2]
    elif p2==0:
        return [p1]
    else:
        return [p1,p2]

In [1142]:
def liste_prix(card4):
    l_prix = []
    if isinstance(card4, list):
        for x in card4:
            if '€' in x and len(x)<30:
                x=x[:-2] #we remove the €
                x=x.replace(' ','') #we remove the spaces
                l_prix=l_prix+traitement_tarif(x.replace(',','.'))       
    return l_prix

In [1143]:
data_combined['Liste_prix'] = data_combined['card_4'].apply(liste_prix)
data_combined['Nb_prix'] = data_combined['Liste_prix'].apply(lambda x: len(x))
data_combined['Prix_min'] = data_combined['Liste_prix'].apply(lambda x: np.min(x) if len(x)>0 else np.nan)
data_combined['Prix_max'] = data_combined['Liste_prix'].apply(lambda x: np.max(x) if len(x)>0 else np.nan)

In [1144]:
data_combined.drop(columns=['URL','card_1','card_2','Liste_prix'],inplace=True)
data_combined.head()

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,moyens_paiement,carte_vitale,secteur,Présentation,Langues parlées,Diplômes nationaux et universitaires,Autres formations,Expériences,Travaux et publications,Prix et distinctions,site,Site web,Formations,Diplômes nationaux et universitaires b,Nb caractères présentation,Autres formations b,Travaux et publications b,Nb langues,Expériences b,Formations b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV,Nb_prix,Prix_min,Prix_max
0,Dr Marian AGACHI,Neurochirurgien,"[Centre Floréal, Lieu 2, Lieu 3, Centre médico...","[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1,0,0,0,Le docteur Marian Agachi vous accueille dans s...,"Anglais, Français et Roumain",,,,,,1,,2012Diplôme Universitaire de Chirurgie Mini-In...,0,748,0,0,3,0,1,154,1,0,1,1,0,0,NaN,NaN
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10,1,1,2,Située au cœur du seizieme arrondissement de ...,Anglais et Français,2011D.I.U. Médecine morphologique et anti-âge ...,2013Implants de cheveux synthétiques - Médicap...,Depuis 1987Cabinet - Paris1984 - 1985Interne -...,2011Intérêt des Light Emitting Diodes dans la ...,,1,,NaN,1,2538,1,1,2,1,0,180,1,0,1,0,0,26,50.0,1000.00
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3,1,1,0,Le docteur Gabriel Ohana vous accueille dans s...,Anglais,2003Diplôme d'État de docteur en chirurgie den...,2005Diplôme Universitaire de Parodontie - Pari...,,,,1,,NaN,1,481,1,0,1,0,0,240,1,0,1,1,0,7,23.0,700.00
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,0,1,1,1,Le docteur Thomas Bamberger vous accueille dan...,Anglais,1994Diplôme d'État de docteur en médecine - Un...,,Interne - Hôpitaux de Paris - ParisAssociation...,,,0,NaN,NaN,1,407,0,0,1,1,0,155,1,0,1,1,0,6,31.5,94.11
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,0,1,1,4,Le docteur Layla Chatila vous accueille dans s...,Anglais,2009C.E.S. Chirurgie dentaire - Prothèse fixée...,,,,,0,NaN,NaN,1,246,0,0,1,0,0,202,1,1,1,1,0,1,23.0,23.00


In [1145]:
def nb_terms(card):
    if type(card)==list :
        return (len(card))
    else :
        return(0)

def nb_carac(card):
    if type(card)==list :
        s = 0
        for item in card : 
            s+=len(card)
        return(s)
    else :
        return(0)

In [1146]:
data_combined['nb terms card_0']=data_combined['card_0'].apply(nb_terms)
data_combined['nb terms card_3']=data_combined['card_3'].apply(nb_terms)
data_combined['nb terms card_4']=data_combined['card_4'].apply(nb_terms)
data_combined['nb carac card_0']=data_combined['card_0'].apply(nb_carac)
data_combined['nb carac card_3']=data_combined['card_3'].apply(nb_carac)
data_combined['nb carac card_4']=data_combined['card_4'].apply(nb_carac)

# Step 3: We separate the doctor whom we know the RPPS from the others  

In [1147]:
indices_nan = data_combined[data_combined['RPPS'].isna()].index.tolist()
print(indices_nan, len(indices_nan))

[8, 23, 27, 73, 74, 94, 112, 116, 117, 118, 121, 122, 123, 124, 127, 130, 131, 132, 135, 136, 138, 139, 140, 141, 142, 143, 145, 148, 150, 151, 152, 153, 156, 157, 158, 160, 161, 163, 166, 167, 169, 170, 172, 173, 175, 176, 177, 178, 179, 180, 182, 183, 184, 186, 187, 188, 190, 191, 192, 194, 196, 197, 198, 201, 202, 204, 209, 213, 214, 215, 216, 219, 220, 221, 222, 223, 225, 226, 227, 229, 230, 231, 233, 234, 235, 236, 238, 239, 240, 241, 245, 246, 247, 249, 250, 252, 254, 256, 259, 260, 262, 263, 264, 265, 267, 268, 270, 271, 272, 276, 277, 278, 279, 280, 281, 282, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 312, 315, 316, 317, 318, 319, 320, 321, 323, 324, 325, 326, 327, 328, 329, 331, 335, 337, 339, 344, 346, 347, 348, 349, 350, 351, 352, 353, 354, 356, 358, 359, 360, 365, 367, 368, 371, 372, 373, 374, 376, 378, 379, 380, 381, 382, 383, 384, 387, 388, 390, 392, 393, 395, 396, 397, 398, 399, 400, 4

In [1148]:
data['Profile Cards'].iloc[999]

'{\'card_0\': \'Tarifs et remboursement\\n\\n\\n\\nConventionné\\nCarte Vitale acceptée\\nTiers payant : Sécurité sociale\\nVoir les tarifs\\nMoyens de paiement\\nChèques, espèces et carte bancaire\\nExpertises et actes\\nBilan podologique\\nIonophorèse\\nK-Tape\\nKinesio Taping\\nOrthonyxie\\nOrthoplastie\\nOrthèse plantaire\\nPodologie\\nPédicurie\\nSemelles orthopédiques\\nVerrue\', \'card_1\': "Carte et informations d\'accès\\n9 Avenue Ampère, 77420 Champs-sur-Marne\\nMoyens de transport\\nBus - Ampere (lignes 213 et 312)\\nBus - Noisy-Champs Rer - Descartes (ligne 212)\\nInformations pratiques\\n1er étage avec ascenseur\\nEntrée accessible\\nJ\'autorise le traitement d\'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus\\nsur la collecte et le traitement des données par Google\\nAFFICHER LA CARTE", \'card_2\': "Présentation\\nDepuis janvier 2010, je vous reçois dans mon cabinet à Champs-sur-Marne à deux pas de

In [1149]:
data_combined_sans_nan = data_combined[data_combined['RPPS'].notna()]

In [1150]:
gen_nan=data_combined[(data_combined['RPPS'].isna()) & (data_combined['Specialty']=='Médecin généraliste')]

In [1151]:
gen_nan.head()

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,moyens_paiement,carte_vitale,secteur,Présentation,Langues parlées,Diplômes nationaux et universitaires,Autres formations,Expériences,Travaux et publications,Prix et distinctions,site,Site web,Formations,Diplômes nationaux et universitaires b,Nb caractères présentation,Autres formations b,Travaux et publications b,Nb langues,Expériences b,Formations b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV,Nb_prix,Prix_min,Prix_max,nb terms card_0,nb terms card_3,nb terms card_4,nb carac card_0,nb carac card_3,nb carac card_4
139,Dr Olivier Gouget,Médecin généraliste,[],NaN,NaN,48.832450,2.241246,"[Expertises et actes, Homéopathie]",NaN,1,0,0,0,,,,,,,,0,NaN,NaN,0,0,0,0,0,0,0,213,1,1,0,0,0,0,NaN,NaN,1,0,0,1,0,0
141,Dr Elisabeth Millara,Médecin généraliste,[],NaN,NaN,48.842998,2.243123,NaN,NaN,0,0,0,0,,,,,,,,0,NaN,NaN,0,0,0,0,0,0,0,201,1,1,1,0,0,0,NaN,NaN,1,0,0,1,0,0
148,Dr Pierre-André Vacquier,Médecin généraliste,[],NaN,NaN,48.832450,2.241246,"[Expertises et actes, Homéopathie]",NaN,1,0,0,0,,,,,,,,0,NaN,NaN,0,0,0,0,0,0,0,191,1,1,0,0,0,0,NaN,NaN,1,0,0,1,0,0
178,Dr Nael MOUSSA,Médecin généraliste,[],NaN,NaN,48.822032,2.305851,NaN,NaN,0,0,0,0,,,,,,,,0,NaN,NaN,0,0,0,0,0,0,0,166,1,0,1,0,0,0,NaN,NaN,1,0,0,1,0,0
183,Dr James SCHINAZI,Médecin généraliste,[],NaN,NaN,48.864290,2.286304,"[Expertises et actes, Médecine esthétique]",NaN,1,0,0,0,,,,,,,,0,NaN,NaN,0,0,0,0,0,0,0,233,1,1,0,0,0,0,NaN,NaN,1,0,0,1,0,0


In [1152]:
data['Profile Cards'][data['Doctor Name']=='Dr Elisabeth Millara']

141    {'card_0': 'Tarifs et remboursement\n\nCarte V...
Name: Profile Cards, dtype: object

In [1153]:
data_combined_sans_nan['Specialty'].value_counts()

Specialty
Médecin généraliste         4804
Ostéopathe                  4507
Chirurgien-dentiste         4154
Psychologue                 2475
Masseur-kinésithérapeute    2155
                            ... 
Cabinet de chiropraxie         1
Psychanalyse                   1
Phoniatre                      1
Mammographie                   1
Hépatite virale                1
Name: count, Length: 128, dtype: int64

# Step 4: We open the data from “Annuaire Santé”  

In [1154]:
annuaire = pd.read_csv('data/sample.csv')
annuaire.shape

(132396, 27)

In [1155]:
#To verify our database
annuaire.isnull().sum()

identifiantpp                            0
identificationnationalepp                0
codecivilité                             0
nomdexercice                             1
prénomdexercice                          1
codecatégorieprofessionnelle             0
codesavoirfaire                          0
libellésavoirfaire                       0
codemodeexercice                         0
numérosiretsite                      79484
numérofinesssite                     88235
identifianttechniquedelastructur     18630
raisonsocialesite                    18634
numérovoiecoordstructure             33476
indicerépétitionvoiecoordstructu    125904
codetypedevoiecoordstructure         89667
libellétypedevoiecoordstructure      89667
libellévoiecoordstructure            20525
bureaucedexcoordstructure            18630
codepostalcoordstructure             18630
codecommunecoordstructure            18638
libellécommunecoordstructure         18638
libellépayscoordstructure            62797
codesecteur

In [1156]:
annuaire['identifiantpp']=annuaire['identifiantpp'].astype(str)

# Step 5: We focus on the general practitioners

In [1157]:
medecin_gen = data_combined_sans_nan[data_combined_sans_nan['Specialty']=='Médecin généraliste'] #4804 médecins généralistes

In [1158]:
#we look for all the general practitioners who are not in the directory 'annuaire' but are in Doctolib
liste_annuaire_manquant=[]
liste_index_annuaire_manquant=[]
for i in range(medecin_gen.shape[0]):
    if medecin_gen['RPPS'].iloc[i] not in annuaire['identifiantpp'].tolist():
        liste_annuaire_manquant.append(medecin_gen['RPPS'].iloc[i])
        liste_index_annuaire_manquant.append(i)


In [1159]:
medecin_gen.reset_index(inplace=True)

In [1160]:
medecin_gen_annuaire = medecin_gen.drop(index=liste_index_annuaire_manquant)

In [1161]:
medecin_gen.shape[0]-medecin_gen_annuaire.shape[0]

147

In [1162]:
medecin_gen_annuaire.shape #4657

(4657, 46)

In [1163]:
liste_med = annuaire.merge(medecin_gen_annuaire,how='outer',left_on='identifiantpp',right_on='RPPS')

# Step 6: We remove duplicates

In [1164]:
liste_med['identifiantpp'].value_counts()

identifiantpp
10100072189    32
10100530905    15
10001930451    15
10100853000    15
10002808433    15
               ..
10003112165     1
10003112058     1
10003112041     1
10003112009     1
10110318770     1
Name: count, Length: 102201, dtype: int64

In [1165]:
liste_med=liste_med[liste_med['codecommunecoordstructure'].notnull()]

In [1166]:
liste_med.reset_index(inplace=True)

In [1167]:
len(liste_med['identifiantpp'].unique())

91903

In [1168]:
dico={}
liste_a_drop=[]
for i in range(liste_med.shape[0]):
    RPPS=liste_med['identifiantpp'].iloc[i]
    codcom=liste_med['libellécommunecoordstructure'].iloc[i]
    if RPPS not in dico.keys():
        dico[RPPS]=[codcom]
    elif codcom not in dico[RPPS]:
        dico[RPPS].append(codcom)
    else:
        liste_a_drop.append(i)
    

In [1169]:
liste_med_sans_doublons=liste_med.drop(index=liste_a_drop)

In [1170]:
#we count the number of general practicioners on Doctolib, it has augmented because of the duplicates in different cities
liste_med_sans_doublons[liste_med_sans_doublons['RPPS'].notnull()].shape[0]

5307

In [1171]:
liste_med_sans_doublons['identifiantpp'].value_counts()

identifiantpp
10100072189    24
10100530905    14
10101351111    10
10003175840    10
10003131389    10
               ..
10002984440     1
10002984317     1
10002984218     1
10002984200     1
10110318770     1
Name: count, Length: 91903, dtype: int64

In [1172]:
def departement(codepostal):
    code = int(codepostal)/1000
    if int(code) == 97 :
        return(str(int(codepostal/100)))
    if int(code) < 10 :
        return('0'+str(int(code)))
    return (str(int(code)))

In [1173]:
liste_med_sans_doublons.loc[:, 'code dépar'] = liste_med_sans_doublons['codepostalcoordstructure'].apply(departement)

# Step 7: A first measure of competition 

In [1174]:
concu = pd.read_excel('data/TCRD_068.xlsx')
concu.rename(columns={'Professionnels de santé au 1ᵉʳ janvier 2023 : comparaisons départementales' : 'Code Département','Unnamed: 1' : 'Nom Département','Unnamed: 2':'Nombre de médecins','Unnamed: 3':'Nombre de médecins pour 100 000 habitants','Unnamed: 4':'généralistes','Unnamed: 5':'spécialistes','Unnamed: 6':'dentistes','Unnamed: 7':'Pharm.'}, inplace=True)
concu.drop(labels = [0,1,2,3,108],axis=0, inplace=True)
concu

,Code Département,Nom Département,Nombre de médecins,Nombre de médecins pour 100 000 habitants,généralistes,spécialistes,dentistes,Pharm.
4,01,Ain,1162,174,99,75,53,78
5,02,Aisne,1107,211,99,111,45,91
6,03,Allier,917,275,134,141,51,111
7,04,Alpes-de-Haute-Provence,483,291,165,125,54,103
8,05,Hautes-Alpes,705,503,291,213,72,135
...,...,...,...,...,...,...,...,...
103,972,Martinique,1171,332,152,180,68,99
104,973,Guyane,717,242,123,120,30,46
105,974,La Réunion,3170,364,176,189,67,94
106,976,Mayotte,265,89,49,39,9,31


In [1175]:
#when we have the good webscraping, we will merge in a new dataframe as we do not need this measure of competition
liste_med_sans_doublons = liste_med_sans_doublons.merge(concu[['Code Département','généralistes']], left_on='code dépar', right_on='Code Département',how='inner')

In [1176]:
liste_med_sans_doublons['identifiantpp'].value_counts().sum()-len(liste_med_sans_doublons['identifiantpp'].unique())

14359

In [1177]:
liste_med_sans_doublons['libellécommunecoordstructure'][liste_med_sans_doublons['identifiantpp']=='10101354222']

91436                   Auxonne
91437    Château-Chinon (Ville)
91438                   Chenôve
91439                     Dijon
91440             Saint-Clément
91441                    Nevers
91442       Châtillon-sur-Seine
Name: libellécommunecoordstructure, dtype: object

# Step 8: A second measure of competition 

In [1178]:
def nom_voie(nom_voie):
    if isinstance(nom_voie,str):
        if nom_voie[0] in ['1','2','3','4','5','6','7','8','9']:
            nom_voie = nom_voie[1:]
        if 'Imp ' in nom_voie:
            return nom_voie.replace('Imp ','Impasse ').lower()
        else:
            return nom_voie.lower()
    else:
        return nom_voie.lower()

In [1179]:
liste_med_ban=liste_med_sans_doublons[liste_med_sans_doublons['libellépayscoordstructure']!='Belgique']

In [1180]:
def fct_voie(type_voie,nom_voie):
    result = ''
    if isinstance(type_voie,str) and isinstance(nom_voie,str):
        result = (type_voie+' '+nom_voie).lower()
    elif isinstance(nom_voie,str):
        result = nom_voie.lower()
    else:
        result = nom_voie
    if isinstance(result,str) and result[0]=='-':
        if result[1]==' ':
            result= result[2:]
        else :
            result = result[1:]
    if isinstance(result,str) and result[0]=='/':
        result = result[1:]
    return result

In [1181]:
liste_med_ban['Voie'] = liste_med_ban.apply(lambda x: fct_voie(x['libellétypedevoiecoordstructure'],x['libellévoiecoordstructure']),axis=1)

In [1182]:
#for the errors in the merge
def codecom3(codecom1):
    if str(codecom1).startswith('75'):
        return('75056')
    elif str(codecom1).startswith('693'):
        return('69123')
    elif str(codecom1).startswith('130') or str(codecom1).startswith('132'):
        return('13055')
    elif str(codecom1).startswith('74011') or str(codecom1).startswith('74268')or str(codecom1).startswith('74093') or str(codecom1).startswith('74217'):
        return('74010')
    elif str(codecom1).startswith('49199'):
        return('52400')
    elif str(codecom1).startswith('29052'):
        return('29003')
    elif str(codecom1).startswith('85060'):
        return('85194')
    elif str(codecom1).startswith('50602') or str(codecom1).startswith('50173') or str(codecom1).startswith('50203'):
        return('50129')
    elif str(codecom1).startswith('49285'):
        return('49301')
    elif str(codecom1).startswith('74181'):
        return('74112')
    elif str(codecom1).startswith('85107'):
        return('85146')
    elif str(codecom1).startswith('85166'):
        return('85194')
    elif str(codecom1).startswith('49375'):
        return('49023')
    elif str(codecom1).startswith('49065'):
        return('49080')
    elif str(codecom1).startswith('91182'):
        return('91228')
    elif str(codecom1).startswith('85069'):
        return('85008')
    elif str(codecom1).startswith('49276'):
        return('49244')
    elif str(codecom1).startswith('22038'):
        return('22206')
    elif str(codecom1).startswith('77491'):
        return('77316')
    elif str(codecom1).startswith('49273'):
        return('49301')
    else:
        return(str(codecom1))

In [1183]:
liste_med_ban['codecommunecoordstructure3']=liste_med_ban['codecommunecoordstructure'].apply(codecom3)

In [1184]:
liste_med_ban_metr=liste_med_ban[~(liste_med_ban['codecommunecoordstructure3'].str.startswith('97'))]

In [1185]:
apl=pd.read_excel('data/APL_mg_2022.xlsx')

In [1186]:
apl.drop(columns=["APL aux médecins généralistes (sans borne d'âge)","APL aux médecins généralistes de 62 ans et moins"],inplace=True)

In [1187]:
apl.head()

,Code commune,Libellé de la commune,Population totale,APL aux médecins généralistes de 65 ans et moins,Population standardisée par la consommation de soins par tranche d'âge
0,01001,L'Abergement-Clémenciat,806.0,1.823,817.600
1,01002,L'Abergement-de-Varey,262.0,2.008,253.836
2,01004,Ambérieu-en-Bugey,14288.0,2.866,14028.085
3,01005,Ambérieux-en-Dombes,1782.0,3.922,1745.171
4,01006,Ambléon,113.0,0.799,120.879


In [1188]:
apl=apl[~(apl['Code commune'].str.startswith('97'))]

In [1189]:
apl['Code commune2'] = apl['Code commune'].apply(codecom3)

In [1190]:
moyM=apl[apl['Code commune2']=='13055']['APL aux médecins généralistes de 65 ans et moins'].mean()
moyL=apl[apl['Code commune2']=='69123']['APL aux médecins généralistes de 65 ans et moins'].mean()
moyP=apl[apl['Code commune2']=='75056']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy74010=apl[apl['Code commune2']=='74010']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy52400=apl[apl['Code commune2']=='52400']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy29003=apl[apl['Code commune2']=='29003']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy85194=apl[apl['Code commune2']=='85194']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy50129=apl[apl['Code commune2']=='50129']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy49301=apl[apl['Code commune2']=='49301']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy74112=apl[apl['Code commune2']=='74112']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy85146=apl[apl['Code commune2']=='85146']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy85194=apl[apl['Code commune2']=='85194']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy49023=apl[apl['Code commune2']=='49023']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy49080=apl[apl['Code commune2']=='49080']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy91228=apl[apl['Code commune2']=='91228']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy85008=apl[apl['Code commune2']=='85008']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy49244=apl[apl['Code commune2']=='49244']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy22206=apl[apl['Code commune2']=='22206']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy77316=apl[apl['Code commune2']=='77316']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy49301=apl[apl['Code commune2']=='49301']['APL aux médecins généralistes de 65 ans et moins'].mean()


In [1191]:
def new_apl(apl,code):
    if code=='13055':
        return moyM
    elif code=='69123':
        return moyL
    elif code=='75056':
        return moyP
    elif code=='74010':
        return moy74010
    elif code=='52400':
        return moy52400
    elif code=='29003':
        return moy29003
    elif code=='85194':
        return moy85194
    elif code=='50129':
        return moy50129
    elif code=='49301':
        return moy49301
    elif code=='74112':
        return moy74112
    elif code=='85146':
        return moy85146
    elif code=='85194':
        return moy85194
    elif code=='49023':
        return moy49023
    elif code=='49080':
        return moy49080
    elif code=='91228':
        return moy91228
    elif code=='85008':
        return moy85008
    elif code=='49244':
        return moy49244
    elif code=='22206':
        return moy22206
    elif code=='77316':
        return moy77316
    elif code=='49301':
        return moy49301
    else:
        return apl

In [1192]:
apl["APL"]=apl.apply(lambda x : new_apl(x['APL aux médecins généralistes de 65 ans et moins'],x['Code commune2']),axis=1)

In [1193]:
apl_unique=apl.drop_duplicates(subset=['Code commune2'])
apl_unique.head() 

,Code commune,Libellé de la commune,Population totale,APL aux médecins généralistes de 65 ans et moins,Population standardisée par la consommation de soins par tranche d'âge,Code commune2,APL
0,01001,L'Abergement-Clémenciat,806.0,1.823,817.600,01001,1.823
1,01002,L'Abergement-de-Varey,262.0,2.008,253.836,01002,2.008
2,01004,Ambérieu-en-Bugey,14288.0,2.866,14028.085,01004,2.866
3,01005,Ambérieux-en-Dombes,1782.0,3.922,1745.171,01005,3.922
4,01006,Ambléon,113.0,0.799,120.879,01006,0.799


In [1194]:
liste_med_ban_apl = liste_med_ban_metr.merge(apl_unique,how='inner', left_on='codecommunecoordstructure3', right_on='Code commune2')

In [1195]:
#Compute the errors' ratio
(liste_med_ban_metr.shape[0]-liste_med_ban_apl.shape[0])/liste_med_ban_metr.shape[0]

0.002183278426480048

In [1196]:
#To find where the errors are
liste_med_ban_apl_outer = liste_med_ban_metr.merge(apl_unique, how='outer', left_on='codecommunecoordstructure3', right_on='Code commune2', indicator=True)
#Filter lines without any match
liste_med_ban_no_match = liste_med_ban_apl_outer[liste_med_ban_apl_outer['_merge'] == 'left_only']
apl_no_match = liste_med_ban_apl_outer[liste_med_ban_apl_outer['_merge'] == 'right_only']

In [1197]:
liste_med_ban_no_match['codecommunecoordstructure3'].value_counts()

codecommunecoordstructure3
59540    23
49350     7
74182     7
29203     5
49238     4
         ..
35303     1
50515     1
28115     1
28101     1
00000     1
Name: count, Length: 102, dtype: int64

# Step 9: We select an instrumental variable 

In [1198]:
fibre=pd.read_excel('data/fibre.xlsx')

c:\miniconda\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\miniconda\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [1199]:
fibre.head()

,Code Insee,Commune,Nombre de locaux,Fibre
0,25008,Aibre,214,0.981308
1,56194,Rieux,1892,0
2,70361,Montessaux,92,0.75
3,25262,Fuans,295,0.928814
4,89168,Fleys,167,0


In [1200]:
fibre.drop(columns=['Nombre de locaux','Commune'],inplace=True)

In [1201]:
#First attempt to merge
liste_med_ban_fibre_outer = liste_med_ban.merge(fibre, how='outer', left_on='codecommunecoordstructure3', right_on='Code Insee', indicator=True)
#Filter the lines without any match in the data frame
liste_med_ban_fibre_no_match = liste_med_ban_fibre_outer[liste_med_ban_fibre_outer['_merge'] == 'left_only']
fibre_no_match = liste_med_ban_fibre_outer[liste_med_ban_fibre_outer['_merge'] == 'right_only']

In [1202]:
liste_med_ban_fibre_no_match.shape

(245, 82)

In [1203]:
liste_med_ban_fibre = liste_med_ban_apl.merge(fibre, how='inner', left_on='codecommunecoordstructure3', right_on='Code Insee')

In [1204]:
liste_med_ban_fibre.shape

(102374, 88)

# Step 10 : We select another instrumental variable : standard of living

In [1205]:
#commune = pd.read_csv('data/FRANCE_COMMUNE_NIVEAU_DE_VIE-FIGARO.csv', sep=';')

# Step 11: We compute the percentage of doctors registered in Doctolib and the percentage of female doctors

In [1206]:
total_medecins = liste_med_ban_fibre.groupby('codecommunecoordstructure3')['identifiantpp'].count()

# Étape 2 : Calculer le nombre de femmes médecins par commune
femmes_medecins = liste_med_ban_fibre[liste_med_ban_fibre['codecivilité'] == 'MME'].groupby('codecommunecoordstructure3')['identifiantpp'].count()
pas_Doctolib=liste_med_ban_fibre[liste_med_ban_fibre['RPPS'].isnull()].groupby('codecommunecoordstructure3')['identifiantpp'].count()
# Étape 3 : Calculer la part de femmes (proportion)
part_femmes = (femmes_medecins / total_medecins).fillna(0)
share_Doctolib=1-(pas_Doctolib/total_medecins).fillna(0)

# Afficher le résultat
type(part_femmes)

pandas.core.series.Series

In [1207]:
liste_med_ban_fibre['share_F'] = liste_med_ban_fibre['codecommunecoordstructure3'].map(part_femmes)
liste_med_ban_fibre['share_Doctolib'] = liste_med_ban_fibre['codecommunecoordstructure3'].map(share_Doctolib)
liste_med_ban_fibre.head()

,level_0,identifiantpp,identificationnationalepp,codecivilité,nomdexercice,prénomdexercice,codecatégorieprofessionnelle,codesavoirfaire,libellésavoirfaire,codemodeexercice,numérosiretsite,numérofinesssite,identifianttechniquedelastructur,raisonsocialesite,numérovoiecoordstructure,indicerépétitionvoiecoordstructu,codetypedevoiecoordstructure,libellétypedevoiecoordstructure,libellévoiecoordstructure,bureaucedexcoordstructure,codepostalcoordstructure,codecommunecoordstructure,libellécommunecoordstructure,libellépayscoordstructure,codesecteurdactivité,libellésecteurdactivité,coderôle,libellérôle,index,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,moyens_paiement,carte_vitale,secteur,Présentation,Langues parlées,Diplômes nationaux et universitaires,Autres formations,Expériences,Travaux et publications,Prix et distinctions,site,Site web,Formations,Diplômes nationaux et universitaires b,Nb caractères présentation,Autres formations b,Travaux et publications b,Nb langues,Expériences b,Formations b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV,Nb_prix,Prix_min,Prix_max,nb terms card_0,nb terms card_3,nb terms card_4,nb carac card_0,nb carac card_3,nb carac card_4,code dépar,Code Département,généralistes,Voie,codecommunecoordstructure3,Code commune,Libellé de la commune,Population totale,APL aux médecins généralistes de 65 ans et moins,Population standardisée par la consommation de soins par tranche d'âge,Code commune2,APL,Code Insee,Fibre,share_F,share_Doctolib
0,39,10000010107,810000010107,MME,EMMANUEL-POINCELOT,Georgette,C,SM26,Qualifié en Médecine Générale,S,NaN,330799784,F330799784,EHPAD RESIDENCE LE VERGER D'ANNA,8.0,NaN,R,Rue,DU GRAND JEANNOT,33350 STE TERRE,33350.0,33485,Sainte-Terre,NaN,SA17,Etab. Personnes Agées,FON-AU,Fonction non définie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,33,180,rue du grand jeannot,33485,33485,Sainte-Terre,1904.0,3.141,1978.641,33485,3.1410,33485,0.616172,0.666667,0.000000
1,43,10000010867,810000010867,M,TACONET,CHRISTIAN,C,SM53,Spécialiste en Médecine Générale,L,NaN,NaN,R10000000618793,CABINET DU DR CHRISTIAN TACONET,80.0,NaN,NaN,NaN,AVENUE MARTIN LUTHER KING,95490 VAUREAL,95490.0,95637,Vauréal,France,SA08,Cabinet de groupe,FON-01,Titulaire de cabinet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95,95,95,avenue martin luther king,95637,95637,Vauréal,16020.0,1.525,14750.708,95637,1.5250,95637,0.994431,0.750000,0.083333
2,50,10000011808,810000011808,MME,HAMOT,ENNA,C,SM26,Qualifié en Médecine Générale,S,7.500959e+13,750831182,F750831182,CDS MEDICAL PARIS REPUBLIQUE,36.0,NaN,QU,Quai,DE JEMMAPES,75010 PARIS,75010.0,75110,Paris 10e Arrondissement,NaN,SA05,Centres de santé,FON-29,Médecin du travail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,75,234,quai de jemmapes,75056,75101,NaN,16030.0,4.997,15591.626,75056,4.1504,75056,0.978945,0.534462,0.054452
3,54,10000012129,810000012129,M,DIDDEN,JEAN-PAUL,C,SM53,Spécialiste en Médecine Générale,L,NaN,NaN,R10100000269875,CABINET DU DR JEAN-PAUL DIDDEN,23.0,NaN,NaN,NaN,AVENUE DE LA JONQUIERE,11500 QUILLAN,11500.0,11304,Quillan,France,SA07,Cabinet individuel,FON-01,Titulaire de cabinet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,11,143,avenue de la jonquiere,11304,11304,Quillan,3141.0,2.716,3599.441,11304,2.7160,11304,0.828451,0.166667,0.000000
4,58,10000012558,810000012558,M,CHUBERRE,YVES,C,SM53,Spécialiste en Médecine Générale,L,NaN,NaN,R1000000056

In [1208]:
medecin_gen1 = liste_med_ban_fibre[liste_med_ban_fibre['RPPS'].notnull()]

In [1209]:
medecin_gen1.columns

Index(['level_0', 'identifiantpp', 'identificationnationalepp', 'codecivilité',
       'nomdexercice', 'prénomdexercice', 'codecatégorieprofessionnelle',
       'codesavoirfaire', 'libellésavoirfaire', 'codemodeexercice',
       'numérosiretsite', 'numérofinesssite',
       'identifianttechniquedelastructur', 'raisonsocialesite',
       'numérovoiecoordstructure', 'indicerépétitionvoiecoordstructu',
       'codetypedevoiecoordstructure', 'libellétypedevoiecoordstructure',
       'libellévoiecoordstructure', 'bureaucedexcoordstructure',
       'codepostalcoordstructure', 'codecommunecoordstructure',
       'libellécommunecoordstructure', 'libellépayscoordstructure',
       'codesecteurdactivité', 'libellésecteurdactivité', 'coderôle',
       'libellérôle', 'index', 'Doctor Name', 'Specialty', 'card_0', 'card_3',
       'card_4', 'Latitude', 'Longitude', 'Skills Processed', 'RPPS',
       'Nb Skills', 'moyens_paiement', 'carte_vitale', 'secteur',
       'Présentation', 'Langues parlées',

# Step 12: We merge the standard of living as well as other informations about the townships with the rest of the data base

In [1210]:
commune = pd.read_excel('data/commune_daniel.xlsx')

In [1211]:
commune = commune[5:]
commune=commune[['Chiffres détaillés     -     Comparateur de territoires', 'Unnamed: 1',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 6',
       'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 17', 'Unnamed: 19',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26']]
commune.rename(columns = {'Chiffres détaillés     -     Comparateur de territoires':'CODGEO','Unnamed: 1' :'LIBGEO',
       'Unnamed: 2':'REG', 'Unnamed: 3':'DEP', 'Unnamed: 4':'Population', 'Unnamed: 6':'Superficie', 'Unnamed: 10':'Naissances',
       'Unnamed: 11':'Deces', 'Unnamed: 17':'Ménages', 'Unnamed: 12':'Logements', 'Unnamed: 19':'MED14',
       'Unnamed: 24':'Population_P_actif', 'Unnamed: 25':'Chomeurs', 'Unnamed: 26':'Actifs'},inplace = True)

In [1212]:
commune.describe()

,CODGEO,LIBGEO,REG,DEP,Population,Superficie,Naissances,Deces,Logements,Ménages,MED14,Population_P_actif,Chomeurs,Actifs
count,36689,36689,36689,36689,35868,35868.00,35398,35398,35868,32929,32929.000000,35868,35868,35868
unique,36689,34174,17,100,5724,5286.00,589,445,30098,3611,26931.000000,28795,26070,28441
top,01001,Sainte-Colombe,44,62,142,6.16,1,1,56,54,20653.333333,105,0,44
freq,1,13,5198,895,85,35.00,4590,5233,35,175,9.000000,37,413,52


In [1213]:
print(medecin_gen1.shape)

(5267, 90)


In [1214]:
medecin_gen1=medecin_gen1.merge(commune, left_on='codecommunecoordstructure3', right_on='CODGEO', how='left')

In [1215]:
print(medecin_gen1.shape)

(5267, 104)


# Step 13: Exportation 

In [1216]:
medecin_gen1.to_csv('medecin_gen.csv')
liste_med_ban_fibre.to_csv('liste_med_ban.csv')